In [1]:
"""
In this notebook i try to use RobertaBaseCased pretrained set on imdb dataset.
After 4 fine tuning epoch model does not procide nice results,
but after more 4 epochs model generates good results.
after 4 more epochs it got much much better
"""

%tensorflow_version 2.x
!pip install ernie==0.0.27b0 > /dev/null
#!pip install ernie==0.0.27b0 

In [2]:
from ernie import SentenceClassifier, Models
import pandas as pd

In [3]:
import keras
NUM_WORDS=1000 # only use top 1000 words
INDEX_FROM=3   # word index offset

train,test = keras.datasets.imdb.load_data(num_words=NUM_WORDS, index_from=INDEX_FROM)
train_x,train_y = train
test_x,test_y = test

word_to_id = keras.datasets.imdb.get_word_index()
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
word_to_id["<UNUSED>"] = 3

id_to_word = {value:key for key,value in word_to_id.items()}
print(' '.join(id_to_word[id] for id in train_x[0] ))

Using TensorFlow backend.


1646592/1641221 [==============================] - 0s 0us/step
<START> this film was just brilliant casting <UNK> <UNK> story direction <UNK> really <UNK> the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same <UNK> <UNK> as myself so i loved the fact there was a real <UNK> with this film the <UNK> <UNK> throughout the film were great it was just brilliant so much that i <UNK> the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the <UNK> <UNK> was amazing really <UNK> at the end it was so sad and you know what they say if you <UNK> at a film it must have been good and this definitely was also <UNK> to the two little <UNK> that played the <UNK> of <UNK> and paul they were just brilliant children are often left out of the <UNK> <UNK> i think because the stars that play them all <UNK> up are such a big <UNK> for the whole film but these children a

In [4]:
sentences = []
for ids in train_x:
  sentence = " ".join([id_to_word[id] for id in ids  ])
  sentences.append( sentence )

In [5]:
sub_size = 1000
sub_set = zip(sentences[0:sub_size],train_y[0:sub_size])
df = pd.DataFrame(sub_set)
df.head(5)

,0,1
0,<START> this film was just brilliant casting <...,1
1,<START> big <UNK> big <UNK> bad music and a <U...,0
2,<START> this has to be one of the worst films ...,0
3,<START> the <UNK> <UNK> at <UNK> the <UNK> sor...,1
4,<START> worst <UNK> of my life br br i <UNK> t...,0


In [6]:
classifier = SentenceClassifier(model_name=Models.RobertaBaseCased, max_length=64)
classifier.load_dataset(df, validation_split=0.2)
classifier.fine_tune(epochs=4, learning_rate=2e-5, training_batch_size=32, validation_batch_size=64)

2020-06-20 08:06:51 [WARNING] Gradients do not exist for variables ['tf_roberta_for_sequence_classification/roberta/pooler/dense/kernel:0', 'tf_roberta_for_sequence_classification/roberta/pooler/dense/bias:0'] when minimizing the loss.


2020-06-20 08:06:53 [WARNING] Gradients do not exist for variables ['tf_roberta_for_sequence_classification/roberta/pooler/dense/kernel:0', 'tf_roberta_for_sequence_classification/roberta/pooler/dense/bias:0'] when minimizing the loss.


2020-06-20 08:06:59 [WARNING] Gradients do not exist for variables ['tf_roberta_for_sequence_classification/roberta/pooler/dense/kernel:0', 'tf_roberta_for_sequence_classification/roberta/pooler/dense/bias:0'] when minimizing the loss.


2020-06-20 08:07:01 [WARNING] Gradients do not exist for variables ['tf_roberta_for_sequence_classification/roberta/pooler/dense/kernel:0', 'tf_roberta_for_sequence_classification/roberta/pooler/dense/bias:0'] when minimizing the loss.


25/25 [==============================] - 520s 21s/step - loss: 0.5955 - accuracy: 0.7050 - val_loss: 0.7061 - val_accuracy: 0.6302


In [7]:
sentence = sentences[1001]
probabilities = classifier.predict_one(sentence)
print("sentence : ",sentence)
print(probabilities)

sentence :  <START> i had never seen a <UNK> movie until <UNK> <UNK> <UNK> i had never seen a movie with <UNK> <UNK> in it i've seen <UNK> of movies very few are <UNK> to me i found last of the <UNK> and <UNK> <UNK> <UNK> to be <UNK> so <UNK> the <UNK> as you read this i started watching <UNK> of the <UNK> country on <UNK> just to see who this <UNK> <UNK> was who has been <UNK> by many for <UNK> hollywood i had no idea what i was in for two hours later i <UNK> out of it and <UNK> i'd watched one of the most beautiful women i had ever seen playing a role perfectly <UNK> to her imagine a movie fan in <UNK> having never seen anyone that <UNK> and that <UNK> before you would have to see her again and again the setting was perfect for a girl that <UNK> she was a poor <UNK> couldn't <UNK> the <UNK> english but you had to <UNK> her what a movie time to start my <UNK> <UNK> movie <UNK>
(0.12442545154026509, 0.8755745484597348)


In [8]:
sentence = "I was bored during that 2 hours. Wanted to go out, but just sit for my friends"
probabilities = classifier.predict_one(sentence)
print("sentence : ",sentence)
print(probabilities)

sentence :  I was bored during that 2 hours. Wanted to go out, but just sit for my friends
(0.4269649116111251, 0.573035088388875)


In [9]:
sentence = "I was bored during that 2 hours. I will not watch an other movie of same director again"
probabilities = classifier.predict_one(sentence)
print("sentence : ",sentence)
print(probabilities)

sentence :  I was bored during that 2 hours. I will not watch an other movie of same director again
(0.41311574418831126, 0.5868842558116887)


In [10]:
classifier.fine_tune(epochs=4, learning_rate=2e-5, training_batch_size=32, validation_batch_size=64)

2020-06-20 08:55:32 [WARNING] Gradients do not exist for variables ['tf_roberta_for_sequence_classification_1/roberta/pooler/dense/kernel:0', 'tf_roberta_for_sequence_classification_1/roberta/pooler/dense/bias:0'] when minimizing the loss.


2020-06-20 08:55:34 [WARNING] Gradients do not exist for variables ['tf_roberta_for_sequence_classification_1/roberta/pooler/dense/kernel:0', 'tf_roberta_for_sequence_classification_1/roberta/pooler/dense/bias:0'] when minimizing the loss.


2020-06-20 08:55:40 [WARNING] Gradients do not exist for variables ['tf_roberta_for_sequence_classification_1/roberta/pooler/dense/kernel:0', 'tf_roberta_for_sequence_classification_1/roberta/pooler/dense/bias:0'] when minimizing the loss.


2020-06-20 08:55:42 [WARNING] Gradients do not exist for variables ['tf_roberta_for_sequence_classification_1/roberta/pooler/dense/kernel:0', 'tf_roberta_for_sequence_classification_1/roberta/pooler/dense/bias:0'] when minimizing the loss.


25/25 [==============================] - 518s 21s/step - loss: 0.1946 - accuracy: 0.9400 - val_loss: 0.8565 - val_accuracy: 0.7135


In [11]:
sentence = "I was bored during that 2 hours. I will not watch an other movie of same director again"
probabilities = classifier.predict_one(sentence)
print("sentence : ",sentence)
print(probabilities)

sentence :  I was bored during that 2 hours. I will not watch an other movie of same director again
(0.7693328805428964, 0.23066711945710358)


In [12]:
sentence = "I was bored during that 2 hours. Wanted to go out, but just sit for my friends"
probabilities = classifier.predict_one(sentence)
print("sentence : ",sentence)
print(probabilities)

sentence :  I was bored during that 2 hours. Wanted to go out, but just sit for my friends
(0.6733486263661812, 0.3266513736338188)


In [13]:
classifier.fine_tune(epochs=4, learning_rate=2e-5, training_batch_size=32, validation_batch_size=64)

2020-06-20 09:41:10 [WARNING] Gradients do not exist for variables ['tf_roberta_for_sequence_classification_2/roberta/pooler/dense/kernel:0', 'tf_roberta_for_sequence_classification_2/roberta/pooler/dense/bias:0'] when minimizing the loss.


2020-06-20 09:41:11 [WARNING] Gradients do not exist for variables ['tf_roberta_for_sequence_classification_2/roberta/pooler/dense/kernel:0', 'tf_roberta_for_sequence_classification_2/roberta/pooler/dense/bias:0'] when minimizing the loss.


2020-06-20 09:41:18 [WARNING] Gradients do not exist for variables ['tf_roberta_for_sequence_classification_2/roberta/pooler/dense/kernel:0', 'tf_roberta_for_sequence_classification_2/roberta/pooler/dense/bias:0'] when minimizing the loss.


2020-06-20 09:41:19 [WARNING] Gradients do not exist for variables ['tf_roberta_for_sequence_classification_2/roberta/pooler/dense/kernel:0', 'tf_roberta_for_sequence_classification_2/roberta/pooler/dense/bias:0'] when minimizing the loss.


25/25 [==============================] - 519s 21s/step - loss: 0.0464 - accuracy: 0.9875 - val_loss: 1.4229 - val_accuracy: 0.6719


In [14]:
sentence = "I was bored during that 2 hours. I will not watch an other movie of same director again"
probabilities = classifier.predict_one(sentence)
print("sentence : ",sentence)
print(probabilities)

sentence :  I was bored during that 2 hours. I will not watch an other movie of same director again
(0.9804956806578731, 0.01950431934212686)


In [15]:
sentence = "I was bored during that 2 hours. Wanted to go out, but just sit for my friends"
probabilities = classifier.predict_one(sentence)
print("sentence : ",sentence)
print(probabilities)

sentence :  I was bored during that 2 hours. Wanted to go out, but just sit for my friends
(0.9864195652442301, 0.013580434755769965)
